# A look on evolution of Python dictionaries. 
It is extra-curricular to the book material, but fits with the chapter spirit. These details provide a fascinating insight into the implementation techniques used in CPython. Please look at Raymond Hattinger talk on same subject.

In [1]:
from pprint import pprint

keys = 'guido sarah barry rachel tim'.split()
values1 = 'blue orange green yellow red'.split()
values2 = 'austin dallas tuscon reno portland'.split()
values3 = 'apple banana orange pear peach'.split()
hashes = list(map(abs, map(hash, keys)))
entries = list(zip(hashes, keys, values1))
comb_entries = list(zip(hashes, keys, values1, values2, values3))

## Databases

In [2]:
pprint(comb_entries)

[(8664339436170913282, 'guido', 'blue', 'austin', 'apple'),
 (1459880389773299350, 'sarah', 'orange', 'dallas', 'banana'),
 (1328909732855774812, 'barry', 'green', 'tuscon', 'orange'),
 (3429570129038273801, 'rachel', 'yellow', 'reno', 'pear'),
 (3209648040892702255, 'tim', 'red', 'portland', 'peach')]


This is how databases store their entries. There are four columns primary_key, colour, city, fruit. It was a very space efficient way of storing data and memory in the 60s was a precious resource. Unless some indexing technique is implemented, this database will be characterised by linear search, which is going to be quite slow for larger data sets.

In [3]:
def database_linear_search():
    pprint(list(zip(keys, values1, values2, values3)))

database_linear_search()
# Things were stored as flat files

[('guido', 'blue', 'austin', 'apple'),
 ('sarah', 'orange', 'dallas', 'banana'),
 ('barry', 'green', 'tuscon', 'orange'),
 ('rachel', 'yellow', 'reno', 'pear'),
 ('tim', 'red', 'portland', 'peach')]


## How Lisp would do it

If we wish to improve performance we can store it the way LISP used to as a lists of key, pairs. The association lists occupy more memory as every key is stored three times instead of once.

In [4]:
def association_lists():
    pprint([
        list(zip(keys, values1)),
        list(zip(keys, values2)),
        list(zip(keys, values3)),
    ])
    
association_lists()

[[('guido', 'blue'),
  ('sarah', 'orange'),
  ('barry', 'green'),
  ('rachel', 'yellow'),
  ('tim', 'red')],
 [('guido', 'austin'),
  ('sarah', 'dallas'),
  ('barry', 'tuscon'),
  ('rachel', 'reno'),
  ('tim', 'portland')],
 [('guido', 'apple'),
  ('sarah', 'banana'),
  ('barry', 'orange'),
  ('rachel', 'pear'),
  ('tim', 'peach')]]


## Seperate Chaining

In [5]:
def separate_chaining(n):
    buckets = [[] for _ in range(n)] # We initialize n empty buckets
    for pair in entries:
        h, key, value = pair
        i = h % n # Will choose a bucket by modulo dividing hash
        buckets[i].append(pair) 
    pprint(buckets)
    
separate_chaining(2)

[[(8664339436170913282, 'guido', 'blue'),
  (1459880389773299350, 'sarah', 'orange'),
  (1328909732855774812, 'barry', 'green')],
 [(3429570129038273801, 'rachel', 'yellow'),
  (3209648040892702255, 'tim', 'red')]]


That gives about 25% improved lookup. Some of the keys like `'sarah'` are found really fast, while with others we still need to go through a list of entries.

In [6]:
separate_chaining(4)

[[(1328909732855774812, 'barry', 'green')],
 [(3429570129038273801, 'rachel', 'yellow')],
 [(8664339436170913282, 'guido', 'blue'),
  (1459880389773299350, 'sarah', 'orange')],
 [(3209648040892702255, 'tim', 'red')]]


In [7]:
separate_chaining(7)

[[(3429570129038273801, 'rachel', 'yellow'),
  (3209648040892702255, 'tim', 'red')],
 [(1328909732855774812, 'barry', 'green')],
 [],
 [(1459880389773299350, 'sarah', 'orange')],
 [(8664339436170913282, 'guido', 'blue')],
 [],
 []]


If we throw more memory at this problem by making more buckets we improve the performance. With 7 separate buckets everyone gets found in just one probe. We have exchanged linear time for constant lookup. However, we need to initialize several lists, which need to be overallocated to allow room to grow, while some of them might even be empty. To avoid overallocation we can one big table...

## Open Addressing
Instead of using multiple lists will just employ one big list and nudge the values around it if they would like to occupy same index.

In [8]:
def open_addressing_linear(n):
    table = [None] * n
    for h, key, value in entries:
        i = h % n
        while table[i] is not None:
            print(f'{key!r} collided with {table[i][0]!r}')
            i = (i + 1) % n  # This is nudge the key further in list
        table[i] = (key, value)
    print('\nOpen Addressing Table:\n')
    pprint(table)

In [9]:
open_addressing_linear(8)


Open Addressing Table:

[None,
 ('rachel', 'yellow'),
 ('guido', 'blue'),
 None,
 ('barry', 'green'),
 None,
 ('sarah', 'orange'),
 ('tim', 'red')]


Rachel wanted the same spot as Barry and Sarah. Sometimes this would lead to a catastrophic linear pile-up in which all of a sudden your code would be dog slow. (These pile-ups are 1972)

We can improve the behaviour of open addressing if we fall on random number generator rather than a slight nudge. specifically we should consider congruential number generator `i = 5 * i + 1`

In [10]:
def open_addressing_multihash(n):
    table = [None] * n
    for h, key, value in entries:
        perturb = h
        i = h % n
        while table[i] is not None:
            print(f'{key!r} collided with {table[i][0]!r}')
            i = (5 * i + perturb + 1) % n
            perturb >>= 5
        table[i] = (key, value)
    print('\nOpen Addressing Table Multihash:\n')
    pprint(table)

In [11]:
open_addressing_multihash(8)


Open Addressing Table Multihash:

[None,
 ('rachel', 'yellow'),
 ('guido', 'blue'),
 None,
 ('barry', 'green'),
 None,
 ('sarah', 'orange'),
 ('tim', 'red')]


Performance slowed down but we avoid the catastrophic pileup

If we use more space we avoid collisions which speeds up performance
but use more memory. This implementation was in Python for quite a
while with some size changes.

We can save memory if we start using a separate list to store the positions of keys in the primary table.

## Compact Dict

In [12]:
def compact_and_ordered(n):
    table = [None] * n
    for pos, entry, in enumerate(entries):
        h = perturb = entry[0]
        i = h % n
        while table[i] is not None:
            i = (5 * i + perturb + 1) % n
            perturb >>= 5
        table[i] = pos
    pprint(entries)
    pprint(table)

In [13]:
compact_and_ordered(8)

[(8664339436170913282, 'guido', 'blue'),
 (1459880389773299350, 'sarah', 'orange'),
 (1328909732855774812, 'barry', 'green'),
 (3429570129038273801, 'rachel', 'yellow'),
 (3209648040892702255, 'tim', 'red')]
[None, 3, 0, None, 2, None, 1, 4]


This together with the key-sharing dict makes current 3.6 CPython dict implementation. Key sharing dicts share keys among different instances of the same class.

In [14]:
def shared_and_compact(n):
    'Compact, ordered, and shared'
    table = [None] * n
    for pos, entry in enumerate(comb_entries):
        h = perturb = entry[0]
        i = h % n
        while table[i] is not None:
            i = (5 * i + perturb + 1) % n
            perturb >>= 5
        table[i] = pos
    pprint(comb_entries)
    pprint(table)

In [15]:
shared_and_compact(8)

[(8664339436170913282, 'guido', 'blue', 'austin', 'apple'),
 (1459880389773299350, 'sarah', 'orange', 'dallas', 'banana'),
 (1328909732855774812, 'barry', 'green', 'tuscon', 'orange'),
 (3429570129038273801, 'rachel', 'yellow', 'reno', 'pear'),
 (3209648040892702255, 'tim', 'red', 'portland', 'peach')]
[None, 3, 0, None, 2, None, 1, 4]


What is particularly elegant is that we can now make these dicts a lot sparser while incurring a very little penalty in terms of memory consumed. The additional sparsity costs only 8 bytes and avoids all of the collisions.

In [16]:
shared_and_compact(16)

[(8664339436170913282, 'guido', 'blue', 'austin', 'apple'),
 (1459880389773299350, 'sarah', 'orange', 'dallas', 'banana'),
 (1328909732855774812, 'barry', 'green', 'tuscon', 'orange'),
 (3429570129038273801, 'rachel', 'yellow', 'reno', 'pear'),
 (3209648040892702255, 'tim', 'red', 'portland', 'peach')]
[None,
 None,
 0,
 None,
 None,
 None,
 1,
 None,
 None,
 3,
 None,
 None,
 2,
 None,
 None,
 4]
